In [ ]:
import os
import sys
import numpy as np
import subprocess
from astropy.table import Table
from astropy.time import Time
import lsst.daf.butler as dafButler
from lsst.pipe.base import Pipeline

%matplotlib ipympl

In [ ]:
import lsst.afw.display as afwDisplay

afwDisplay.setDefaultBackend("firefly")
display1 = afwDisplay.Display(frame=1)
display2 = afwDisplay.Display(frame=2)

In [ ]:
# Catalog generation parameters
ra_min, ra_max = 52, 54
dec_min, dec_max = -29, -27
n_galaxies = 1000
sky_area_value = 5.0

# Butler configuration
repo_path = "/repo/main"
instrument = "LSSTComCam"

# Specific exposure and detector to process
visit = 2024110800246
detector = 5

# Collection with pvi images
collection = "LSSTComCam/runs/DRP/DP1/w_2025_10/DM-49359"

# Collections for processing results
inject_collection = "u/rea3/slsim/injection_test10_calexp"
output_collection = "u/rea3/slsim/test10_calexp"

# Filter visits to use for ingestion
filters = ["r"]

# Time based selection of visits
start_time = "2024-11-01T00:00:00"
end_time = "2024-12-29T00:00:00"

ngal = 10000
sky_area = 0.5
output_file = "catalog.fits"

# path to slsim
slsim_path = "../slsim/slsim/LsstSciencePipeline/"

In [ ]:
gen_cmd = (
    f"python {slsim_path}/generate_catalog.py --ra-min {ra_min} --ra-max {ra_max} "
    f" --dec-min {dec_min} --dec-max {dec_max} --start-time {start_time} --end-time {end_time}"
    f" --n-galaxies {ngal} --sky-area {sky_area} --repo-path {repo_path}"
    f" --collection {collection} --instrument {instrument}"
    f" --filters {" ".join(filters)} --output-file {output_file}"
)

In [ ]:
# os.system(gen_cmd)
print(gen_cmd)

In [ ]:
# Use the ingest_injection_catalog command line tool to ingest the previously created catalog
injest_cmd = (
    f"ingest_injection_catalog -b {repo_path} -o {inject_collection}"
    f" --injection-catalog {output_file} {" ".join(filters)} -t injection_slsim"
    f" --format fits"
)

In [ ]:
# os.system(injest_cmd)
print(injest_cmd)

In [ ]:
# Create a pipeline configuration file for SLSim injection with DRP processing
pipeline_yaml = "config_wfake.yaml"

with open(pipeline_yaml, "w") as f:
    f.write(
        """
description: Strong lensing source injection pipeline with DRP processing
instrument: lsst.obs.lsst.LsstComCam

tasks:
  inject_visit_slsim:
    class: slsim.LsstSciencePipeline.inject_slsim_visit.VisitInjectSLSimTask
    config:
      external_psf: false
      external_photo_calib: false
      external_wcs: false
      connections.input_exposure: pvi
      connections.output_exposure: injected_slsim_pvi
      selection: "np.isin(injection_catalog['visit'], {visit})"
  subtractImages:
    class: lsst.ip.diffim.subtractImages.AlardLuptonSubtractTask
    config:
    - connections.coaddName: goodSeeing
      connections.template: goodSeeingDiff_templateExp
      connections.science: injected_slsim_pvi
      connections.difference: injected_slsim_goodSeeingDiff_differenceTempExp
      connections.matchedTemplate: injected_slsim_goodSeeingDiff_matchedExp
  detectAndMeasureDiaSources:
    class: lsst.ip.diffim.detectAndMeasure.DetectAndMeasureTask
    config:
    - connections.coaddName: goodSeeing
      connections.diaSources: injected_slsim_goodSeeingDiff_diaSrc
      connections.subtractedMeasuredExposure: injected_slsim_goodSeeingDiff_differenceExp
      connections.science: injected_slsim_pvi
      connections.difference: injected_slsim_goodSeeingDiff_differenceTempExp
      connections.matchedTemplate: injected_slsim_goodSeeingDiff_matchedExp

subsets:
  injected_DRP:
    subset:
      - inject_visit_slsim
      - subtractImages
      - detectAndMeasureDiaSources
    description: >
      Pipeline for injecting strong lensing sources and running difference imaging
"""
    )

print(f"Created pipeline configuration in {pipeline_yaml}")
print(
    "The pipeline includes strong lensing injection followed by subtraction and detection tasks, exactly matching test.yaml"
)

In [ ]:
run_cmd = (
    f"pipetask run -b {repo_path}"
    f' -d "exposure={visit} and detector={detector}" -p {pipeline_yaml}#injected_DRP'
    f" -i {collection},{inject_collection}"
    f" -o {output_collection} --register-dataset-types"
)

In [ ]:
# os.system(run_cmd)
print(run_cmd)

In [ ]:
butler = dafButler.Butler(repo_path, collections=output_collection)

In [ ]:
injected_pvi = butler.get("injected_slsim_pvi", visit=visit, detector=detector)

In [ ]:
diff_pvi = butler.get(
    "injected_slsim_goodSeeingDiff_differenceTempExp", visit=visit, detector=detector
)

In [ ]:
display1.mtv(injected_pvi)
display2.mtv(diff_pvi)

In [ ]:
output_collection